<a href="https://colab.research.google.com/github/ychoi-kr/llm-api-prog/blob/main/4_openai/openai_assistant_with_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [ ]:
from google.colab import userdata
from openai import OpenAI
import time
import requests
import json

In [ ]:
api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

In [ ]:
# plainkorean.kr의 인증서 관련 경고를 무시
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [ ]:
def get_simpler_alternatives(keyword):
    url = f"https://plainkorean.kr/api.jsp?keyword={keyword}"
    print(f"Requesting: {url}")  # 요청 URL 출력
    response = requests.get(url, verify=False)
    print(f"Response status code: {response.status_code}")  # 응답 상태 코드 출력
    if response.status_code == 200:
        result = response.json()
        print(f"Response content: {result}")  # 응답 내용 출력
        if result:
            return json.dumps(result[0], ensure_ascii=False)
    return json.dumps({"error": "No alternatives found"})

In [ ]:
get_simpler_alternatives_json = {
    "name": "get_simpler_alternatives",
    "description": "Get simpler alternatives for a given Korean keyword from plainkorean.kr API",
    "parameters": {
        "type": "object",
        "properties": {
            "keyword": {
                "type": "string",
                "description": "The Korean keyword to find simpler alternatives for",
            },
        },
        "required": ["keyword"],
    },
}

In [ ]:
assistant = client.beta.assistants.create(
    name="쉬운 말 추천v2",
    instructions="문장에서 어려운 단어를 식별하고, 제공된 함수를 사용해 더 쉬운 표현을 제안하세요.",
    model="gpt-4o",
    tools=[{"type": "function", "function": get_simpler_alternatives_json}],
)

In [ ]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="매니지드 서비스는 제품 설치부터 점검, 운영까지 대신해 준다.",
)

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

In [ ]:
while True:
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id,
    )
    run_status = run.status

    if run_status == "requires_action" and run.required_action is not None:
        tools_to_call = run.required_action.submit_tool_outputs.tool_calls
        tool_output_array = []
        for tool in tools_to_call:
            tool_call_id = tool.id
            function_name = tool.function.name
            function_arg = json.loads(tool.function.arguments)
            if function_name == 'get_simpler_alternatives':
                output = get_simpler_alternatives(function_arg["keyword"])
            tool_output_array.append({"tool_call_id": tool_call_id, "output": output})

        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_output_array,
        )
    elif run_status in ["completed", "failed"]:
        break

    time.sleep(1)

Requesting: https://plainkorean.kr/api.jsp?keyword=매니지드
Response status code: 200
Response content: [{'alt': '위탁 관리 서비스', 'keyword': '매니지드 서비스', 'example': ["(X) 클라우드의 성공은 '매니지드 서비스'가 좌우한다.", "(O) 자원 공유의 성공은 '위탁 관리 서비스'가 좌우한다.", '(X) 매니지드 서비스는 제품 설치부터 점검, 운영까지 대신해 준다.', '(O) 위탁 관리 서비스는 제품 설치부터 점검, 운영까지 대신해 준다.', '(X) 구독 서비스에서 한 단계 확장한 매니지드 서비스도 제공한다. ', '(O) 구독 서비스에서 한 단계 확장한 위탁 관리 서비스도 제공한다.']}]
Requesting: https://plainkorean.kr/api.jsp?keyword=점검
Response status code: 200
Response content: []


In [ ]:
if run_status == 'completed':
    messages = client.beta.threads.messages.list(
        thread_id=thread.id,
    )
    print(messages.data[0].content[0].text.value)
else:
    print(f"Run status: {run_status}")

문장에서 어려운 단어에 대한 더 쉬운 표현을 제안드리겠습니다.

- **매니지드 서비스**: "위탁 관리 서비스"로 표현할 수 있습니다.

"점검"에 대해서는 더 쉬운 대체 표현을 찾을 수 없었습니다. "점검"은 이미 상대적으로 쉽게 이해될 수 있는 단어일 수 있습니다.

따라서 문장을 더 쉽게 표현하면 다음과 같습니다:
- "위탁 관리 서비스는 제품 설치부터 점검, 운영까지 대신해 준다."


In [ ]:
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_kjIXu8axFa1XXTkg4BD7NWix', deleted=True, object='assistant.deleted')